# Testing crawler and site structure

## Testing using `requests` library (non-asyncronous)

In [1]:
import requests
import re

import urllib.parse as urlparse

from bs4 import BeautifulSoup as bs

URL_MAIN = 'https://www.inform.kz'
URL_ARCHIVE = 'https://www.inform.kz/ru/archive'

session = requests.Session()


### Check for the crawling links

#### Check for article links in the first page of archives

In [2]:
url_links = 'https://www.inform.kz/ru/archive?date=01.01.2012'
response = session.get(url_links)
soup = bs(response.content, 'html.parser')

links = soup.find('div', class_='news-list__col').find_all('a')
links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
links = list(links)
print(f'Links for articles (total {len(links)} links) from main page:\n{links[:5]}...')

Links for articles (total 20 links) from main page:
['https://www.inform.kz/ru/kitayskie-analitiki-predskazyvayut-obostrenie-geopoliticheskoy-napryazhennosti-v-2012g_a2430369', 'https://www.inform.kz/ru/samye-ozhidaemye-avtonovinki-2012-goda_a2430350', 'https://www.inform.kz/ru/v-kazahstane-sostavili-reyting-samyh-opasnyh-dorog-smi_a2430384', 'https://www.inform.kz/ru/v-ekibastuze-pri-pozhare-v-zhilom-dome-pogib-muzhchina-esche-odin-gospitalizirovan_a2430360', 'https://www.inform.kz/ru/na-filippinah-ot-pirotehniki-postradali-200-chelovek_a2430344']...


#### Check for pagination links

In [3]:
soup = bs(response.content, 'html.parser')
links = soup.find('p', class_='pagination').find_all('a')
links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
links = list(links)
print(f'Pagination links: {links}')

Pagination links: ['https://www.inform.kz/ru/archive/2?date=01.01.2012']


Check for articles from next page:

In [4]:
response = session.get(links[0])
soup = bs(response.content, 'html.parser')

links = soup.find('div', class_='news-list__col').find_all('a')
links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
links = list(links)
print(f'Links for articles (total {len(links)} links) from next page:\n{links[:5]}...')

Links for articles (total 2 links) from next page:
['https://www.inform.kz/ru/chislo-zhertv-ciklona-v-indii-vozroslo-do-42_a2430340', 'https://www.inform.kz/ru/ya-ot-vsego-serdca-zhelayu-chtoby-v-novom-godu-sbylis-vse-svetlye-mechty-i-nadezhdy-kazhdogo-kazahstanca-pozdravlenie-prezidenta-rk_a2430289']...


### Check for crawling articles

In [5]:
def check_articles(url):
    response = session.get(url)

    soup = bs(response.content, 'html.parser')

    title = soup.find('div', class_='title_article_bl')
    title = re.sub(r'\s+', ' ', title.get_text().strip())

    date = soup.find('div', class_='time_article_bl')
    date = re.sub(r'\s+', ' ', date.get_text().strip())
    
    links = []
    links_frame = soup.find('div', class_='frame_news_article_adapt')
    if links_frame:
        links = links_frame.find_all('a')
        links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
        links = list(links)
        links_frame.decompose()
    
    body = soup.find('div', class_='body_article_bl')
    body = re.sub(r'\s+', ' ', body.get_text().strip())
    
    tags = soup.find('div', class_='keywords_bl')
    if tags:
        tags = tags.find_all('a')
        tags = [re.sub(r'\s+', ' ', t.get_text().strip()) for t in tags]
    else:
        tags = []
    
    author = soup.find('div', class_='data_author_bl')
    if author:
        author = author.find('a')
        author = re.sub(r'\s+', ' ', author.get_text().strip())
    else:
        author = None
    
    print(f'Title: {title}')
    print(f'Date: {date}')
    print(f'Links (number of links - {len(links)}): {links[:3]}')
    print(f'Tags: {tags}')
    print(f'Author: {author}')
    print(f'Body:\n{body}')

In [6]:
url1 = 'https://www.inform.kz/ru/v-zhanaozene-prodolzhaetsya-rabota-po-blagoustroystvu-goroda_a2430452'
url2 = 'https://www.inform.kz/ru/prezident-kazahstana-provel-peregovory-s-general-nym-sekretarem-oon_a3981340'

In [7]:
check_articles(url1)

Title: В Жанаозене продолжается работа по благоустройству города
Date: 2 Января 2012, 13:35
Links (number of links - 0): []
Tags: []
Author: Дамир Байманов
Body:
АСТАНА. 2 января. КАЗИНФОРМ - В г. Жанаозен продолжаются мероприятия по вывозу бытового мусора и благоустройству города, сообщает pm.kz со ссылкой на пресс-службу акимата Мангыстауской областиПо информации пресс-службы, работа по очистке города завершена. В то же время продолжается уборка бытового мусора и благоустройство города. По данным на 1 января, в Жанаозене на уборочных работах с ГКП «Тазалык» задействовано 96 человек и 23 единицы специальной техники, вывезено 1154 тонн мусора.


In [8]:
check_articles(url2)

Title: Президент Казахстана провел переговоры с Генеральным секретарем ООН
Date: 21 Сентября 2022, 05:06
Links (number of links - 3): ['https://www.inform.kz/ru/glava-gosudarstva-podpisal-zakon-po-formirovaniyu-kadrovogo-sostava-eek_a3988798', 'https://www.inform.kz/ru/glava-gosudarstva-vstretil-v-stolichnom-aeroportu-emira-katara_a3989363', 'https://www.inform.kz/ru/prezident-prinyal-general-nogo-sekretarya-odkb_a3988814']
Tags: ['ООН', 'Президент Казахстана']
Author: Ринат Дусумов
Body:
НЬЮ-ЙОРК. КАЗИНФОРМ - Программа рабочего визита Касым-Жомарта Токаева в Нью-Йорк для участия в 77-й сессии Генеральной Ассамблеи ООН завершилась переговорами с Генеральным секретарем ООН Антониу Гутерришем, передает МИА «Казинформ» со ссылкой на пресс-службу Акорды. В ходе встречи, прошедшей в конструктивной и дружественной атмосфере, стороны обсудили состояние и перспективы укрепления многостороннего сотрудничества между Казахстаном и ООН. Особое внимание было уделено усилиям, направленным на выработ

## Testing asyncronous HTTP using `aiohttp` package

In [10]:
import aiohttp
import asyncio

async def _get_url(session, url):
    '''
    Function to perform GET request on the given url using the session object.

    Args:
        session (aiohttp.ClientSession): Session object.
        url (str): URL to fetch.

    Returns:
        str: Response content in string format.
    '''
    async with session.get(url) as response:
        return await response.text()


def retrieve_data(content):
    '''
    Get data from the response content of article.

    Args:
        content (str): String representation of the response content.

    Returns:
        dict: Dictionary object with scraped fields.
    '''
    soup = bs(content, 'html.parser')

    title = soup.find('div', class_='title_article_bl')
    title = re.sub(r'\s+', ' ', title.get_text().strip())

    date = soup.find('div', class_='time_article_bl')
    date = re.sub(r'\s+', ' ', date.get_text().strip())
    
    links = []
    links_frame = soup.find('div', class_='frame_news_article_adapt')
    if links_frame:
        links = links_frame.find_all('a')
        links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
        links = list(links)
        links_frame.decompose()
    
    body = soup.find('div', class_='body_article_bl')
    body = re.sub(r'\s+', ' ', body.get_text().strip())
    
    tags = soup.find('div', class_='keywords_bl')
    if tags:
        tags = tags.find_all('a')
        tags = [re.sub(r'\s+', ' ', t.get_text().strip()) for t in tags]
    else:
        tags = []
    
    author = soup.find('div', class_='data_author_bl')
    if author:
        author = author.find('a')
        author = re.sub(r'\s+', ' ', author.get_text().strip())
    else:
        author = None
    
    return {
        'title': title,
        'date': date,
        'body': body,
        'links': links,
        'tags': tags,
        'author': author
    }

In [ ]:
async with aiohttp.ClientSession() as session:
    # given the date, get the first page
    url_for_date = 'https://www.inform.kz/ru/archive?date=01.04.2022'
    response_main = await _get_url(session, url_for_date)

    # get pagination links
    soup = bs(response_main, 'html.parser')
    links_pages = soup.find('p', class_='pagination').find_all('a')
    links_pages = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links_pages])
    links_pages = list(links_pages)

    response_pages = await asyncio.gather(*[
        _get_url(session, link) for link in links_pages
    ])

    response_pages = [response_main] + response_pages
    print(f'Number of page links: {len(response_pages)}.')

    # get all articles
    links_articles_all = []
    for page in response_pages:
        soup = bs(page, 'html.parser')

        links_articles = soup.find('div', class_='news-list__col').find_all('a')
        links_articles = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links_articles])
        links_articles_all.extend(list(links_articles))
    print(f'Number of artilces: {len(links_articles_all)}.')

    respose_articles = await asyncio.gather(*[
        _get_url(session, link) for link in links_articles_all
    ])

    # get the articles data for single page
    data = []
    for r in respose_articles:
        data.append(retrieve_data(r))

Number of page links: 7.
Number of artilces: 140.
